In [ ]:
import pandas as pd
import folium
from folium import plugins
import json
import geopandas as gp

In [ ]:
#### GeoJson ####
## MultoPolygon
# 경찰서 / 치안안전센터 관할경계 geojson 파일
geo_PO_area = json.load(open('data/geojson/11.4개시도_경찰서_관할경계.geojson', encoding='utf-8'))
geo_SC_area = json.load(open('data/geojson/12.4개시도_치안안전센터_관할경계.geojson', encoding='utf-8'))
#격자 기반
suwon_grid = json.load(open('data/geojson/32.수원시_표준격자(250M_x_250M).geojson', encoding='utf-8'))
suwon_grid_report = json.load(open('data/geojson/2.수원시_112신고(격자별 신고횟수).geojson', encoding='utf-8'))
suwon_grid_SexAge = json.load(open('data/geojson/30.수원시_성연령별_거주인구격자(250M_X_250M).geojson', encoding='utf-8'))
#행정동
suwon_Area = json.load(open('data/geojson/31.수원시_행정경계(읍면동).geojson', encoding='utf-8'))


In [ ]:
df_SC_loc = pd.read_csv('data/csv/39.4개시도_경찰관서현황.csv', encoding='utf-8')
df_Report_loc = pd.read_csv('data/csv/3.3개시도_112신고이력(관할경계매핑).csv', encoding='utf-8') 

In [ ]:
### 4개시도_경찰관할구역_치안센터_MarkerCluster
jsonname = []
jsonlat = []
jsonlng = []
for i in range(len(df_SC_loc)) :
    jsonlng.append(df_SC_loc.iloc[i]['lon'])
    jsonlat.append(df_SC_loc.iloc[i]['lat'])
    jsonname.append(df_SC_loc.iloc[i]['name'])

center = [37.541, 126.986]

# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다.
m2 = folium.Map(location=center,
               tiles='CartoDB dark_matter',
               zoom_start=10)

P_area = folium.Choropleth(
    geo_data = geo_PO_area,
    line_weight = 1,
    line_color = 'yellow',
    fill_opacity = 0, 
    highlight=True,
).add_to(m2)

P_area.geojson.add_child(
    folium.features.GeoJsonTooltip(['NAME'],labels=False)
)

marker_cluster = plugins.MarkerCluster().add_to(m2)

for lat,lng,name in zip(jsonlat,jsonlng,jsonname) :
    folium.Marker(
        location = [lat, lng],
        icon = folium.Icon(color='red', icon='ok'),
        popup = folium.Popup(folium.Html('<div style="text-align:center;font-weight:bold">%s</div>'%name,script=True)
                             , max_width = 100, min_width=100),
    ).add_to(marker_cluster)
    
folium.plugins.MiniMap().add_to(m2)
display(m2)
m2.save('4개시도_경찰관할구역_치안센터_MarkerCluster.html')

In [ ]:
### 4시도 경철관할구역별 치안센터 분포 ###

jsonname = []
jsonlat = []
jsonlng = []
for i in range(len(df_SC_loc)) :
    jsonlng.append(df_SC_loc.iloc[i]['lon'])
    jsonlat.append(df_SC_loc.iloc[i]['lat'])
    jsonname.append(df_SC_loc.iloc[i]['name'])

#fill_color='YlGnBu' 
center = [37.541, 126.986]

# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다.
m = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=11)
P_area = folium.Choropleth(
    geo_data = geo_PO_area,
    line_weight = 1,
    line_color = 'blue',
    fill_opacity = 0, 
    highlight=True,
).add_to(m)

P_area.geojson.add_child(
    folium.features.GeoJsonTooltip(['NAME'],labels=False)
)

for lat,lng,name in zip(jsonlat,jsonlng,jsonname) :
    folium.CircleMarker(
        location = [lat, lng],
        radius = 1,
        color = 'red',
        popup = folium.Popup(folium.Html('<div style="text-align:center;font-weight:bold">%s</div>'%name,script=True)
                             , max_width = 100, min_width=100),
    ).add_to(m)

#folium.LayerControl().add_to(m)
#folium.TileLayer('Mapbox Bright').add_to(m)
display(m)
m.save('4개시도_경찰관할구역_치안센터_분포.html')

In [ ]:
total = df_Report_loc.copy()
danger = df_Report_loc[df_Report_loc['case_type'].isin(['위험방지','기타형사범', '폭력', '시비'])]
noise = df_Report_loc[df_Report_loc['case_type'].isin(['소음','행패소란'])]
traffic = df_Report_loc[df_Report_loc['case_type'].isin(['교통위반', '교통사고', '교통불편'])]

total['count'] = 1
danger['count'] = 1
noise['count'] = 1
traffic['count'] = 1

print(danger.info())

# danger['rec_date'] = danger['rec_date'].astype(str)
# danger['rec_date'] = danger['rec_date'].str.cat(danger['rec_time'])
# danger.drop('rec_time', axis=1, inplace=True)

In [ ]:
# 각 치안센터 신고내역
#danger.drop(['case_type', 'rec_time','arrival_date'],axis=1, inplace=True)

#gp_danger = pd.DataFrame(danger.groupby(['rec_date','psname','name'])['count'].sum().reset_index())
gp_total = pd.DataFrame(total.groupby(['name'])['count'].sum().reset_index())
gp_danger = pd.DataFrame(danger.groupby(['name'])['count'].sum().reset_index())
gp_noise = pd.DataFrame(noise.groupby(['name'])['count'].sum().reset_index())
gp_traffic = pd.DataFrame(traffic.groupby(['name'])['count'].sum().reset_index())

name =[]
for f in geo_SC_area['features'] :
   name.append(f['properties']['NAME'])
print(len(name))

center = pd.DataFrame(name)
center.columns = ['NAME']

mg_total = pd.merge(gp_total, center,how='right',left_on='name',right_on='NAME')
mg_danger = pd.merge(gp_danger , center,how='right',left_on='name',right_on='NAME')
mg_noise = pd.merge(gp_noise , center,how='right',left_on='name',right_on='NAME')
mg_traffic = pd.merge(gp_traffic , center,how='right',left_on='name',right_on='NAME')
#display(gp_total.iloc[1270:1281])
#display(mg_total.iloc[1270:1281])

# print(gp_danger.isnull().sum(axis=0))
# print(gp_noise.isnull().sum(axis=0))
# print(gp_traffic.isnull().sum(axis=0))

mg_total['count'].fillna(0,inplace=True)
mg_danger['count'].fillna(0,inplace=True)
mg_noise['count'].fillna(0,inplace=True)
mg_traffic['count'].fillna(0,inplace=True)

# mg_total.set_index('NAME',inplace=True)
# mg_danger.set_index('NAME',inplace=True)
# mg_noise.set_index('NAME',inplace=True)
# mg_traffic.set_index('NAME',inplace=True)

mg_total['count'] = mg_total['count'].astype(int)
mg_danger['count'] = mg_danger['count'].astype(int)
mg_noise['count'] = mg_noise['count'].astype(int)
mg_traffic['count'] = mg_traffic['count'].astype(int)

# display(mg_total)
# display(mg_danger)
# display(mg_noise)
# display(mg_traffic)
# print(mg_total.info())

jsonname = []
jsonlat = []
jsonlng = []
for i in range(len(df_SC_loc)) :
    jsonlng.append(df_SC_loc.iloc[i]['lon'])
    jsonlat.append(df_SC_loc.iloc[i]['lat'])
    jsonname.append(df_SC_loc.iloc[i]['name'])



center = [37.541, 126.986]

# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다.
m3 = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=11)
data = [mg_total, mg_danger, mg_noise, mg_traffic]
color = ['RdPu', 'YlOrRd', 'YlOrBr', 'YlGnBu' ]
name = ['전체 신고', '폭력/형사범 신고', '소음 신고', '교통 신고' ]

for data, color, name in zip(data,color,name) :
    map=folium.Choropleth(
        geo_data = geo_SC_area,
        data = data,
        columns = ('NAME', 'count'),
        key_on = 'properties.NAME',
        line_weight = 1,
        fill_color = color,
        fill_opacity = 0.3, 
        highlight=True,
        legend_name = name,
        name = name,
    ).add_to(m3)
    
    map.geojson.add_child(
        folium.features.GeoJsonTooltip(['NAME'],labels=True)
    )
    
for lat,lng,name in zip(jsonlat,jsonlng,jsonname) :
    folium.CircleMarker(
        location = [lat, lng],
        radius = 1,
        color = 'red',
        popup = folium.Popup(folium.Html('<div style="text-align:center;font-weight:bold">%s</div>'%name,script=True)
                             , max_width = 100, min_width=100),
    ).add_to(m3)

folium.LayerControl().add_to(m3)
folium.plugins.MiniMap().add_to(m3)

display(m3)

m3.save('4개시도_경찰관할구역_누적신고+치안센터_위치.html')
# display(noise)
# display(traffic)

In [ ]:
gid =[]
rep_cnt = []

#print(len(suwon_grid_report['features']))
#print(len(suwon_grid['features']))
for i in range(len(suwon_grid_report['features'])):
    gid.append(suwon_grid_report['features'][i]['properties']['gid'])
    rep_cnt.append(suwon_grid_report['features'][i]['properties']['rep_cnt'])

grid_rep = pd.DataFrame({'gid':gid,
                        'rep_cnt':rep_cnt}) 
display(grid_rep)
#print(len(grid_rep['gid'].unique()))
#print(max(grid_rep['rep_cnt']))

## 수원시 표준격자 활용
center = [37.280364359333866, 127.00776116595848] # 수원 가운데

suwon_gmap = folium.Map(location=center,
               tiles='cartodbpositron',
               zoom_start=12)

grid_rep_map = folium.Choropleth(
                    geo_data = suwon_grid,
                    data = grid_rep,
                    columns = ('gid', 'rep_cnt'),
                    key_on = 'properties.gid',
                    line_weight = 0.3,
                    fill_color = 'YlOrRd',
                    fill_opacity = 0.5, 
                    legend_name = '격자구역 별 전체 신고접수 빈도',
                    highlight=True,    
                ).add_to(suwon_gmap)

grid_rep_map.geojson.add_child(
    folium.features.GeoJsonTooltip(['gid'], labels= False)
)

dong_area_map = folium.Choropleth(
                    geo_data = suwon_Area,
                    line_weight = 0.3,
                    line_color = 'black',
                    line_opacity = 0.7,
                    fill_opacity = 0, 
                    highlight = True,
                ).add_to(suwon_gmap)

dong_area_map.geojson.add_child(
    folium.features.GeoJsonTooltip(['ADM_DR_NM'],labels=False)
)
folium.LayerControl().add_to(suwon_gmap)
display(suwon_gmap)

suwon_gmap.save('수원시_행정동별_신고접수.html')